In [1]:
from azure.core.credentials import AzureKeyCredential
import openai
from openai.embeddings_utils import cosine_similarity, get_embedding
from azure.search.documents import SearchClient
from azure.search.documents.models import RawVectorQuery
from dotenv import load_dotenv
import os
import json

load_dotenv("./.env");

AZURE_OPENAI_URL = os.getenv("OPENAI_API_BASE_URL")
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE_URI")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
EMBEDDING_MODEL = os.getenv("AZURE_OPENAI_EMB_MODEL_NAME")
COMPLETIONS_MODEL = os.getenv("AZURE_OPENAI_CHATGPT_MODEL")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = str(os.getenv("OPENAI_HOST"))
openai.api_base = AZURE_OPENAI_URL
openai.api_version = os.getenv("OPENAI_API_VERSION")

In [2]:
def get_function_call(messages, function_call = "auto"):
    functions = [
        {
            "name": "get_filters",
            "description": "Get filter for search",
            "parameters": {
                "type": "object",
                "properties": {
                    "year": {
                        "type": "string",
                        "description": "The year of the query",
                    }
                },
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        deployment_id = COMPLETIONS_MODEL,
        messages=messages,
        functions=functions,
        function_call=function_call, 
    )

    return response

def create_filter_expression(json_obj):
    return ' and '.join([f"{k} eq '{v}'" for k, v in json_obj.items()])

def vector_search(search_query, filter):
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, index_name=AZURE_SEARCH_INDEX, credential=credential)

    search_vector = get_embedding(search_query, engine=EMBEDDING_MODEL)
    r = search_client.search(None, top=5, vector_queries=[RawVectorQuery(vector=search_vector, k=5, fields="embedding", exhaustive=True)], filter=filter)

    content = ""
    for doc in r:
        doccontent = doc["content"].replace("\n", " ")
        content += "\n" + f"{doc['sourcepage']}: {doccontent}"
        print(f"{doc['@search.score']}: {doc['sourcepage']}: {doccontent}")
    
    return content

In [3]:
system_chat_template = """
You are an intelligent assistant helping employees with our social memory questions. 
Use 'you' to refer to the individual asking the questions even if they ask with 'I'. 
For tabular information return it as an html table. Do not return markdown format. 
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
Answer always in the user language.
"""

def get_rag_response(search_query):
    messages = []
    messages.append({ "role": "system", "content": system_chat_template })

    prompt = [{"role": "user", "content": search_query}]
    response = get_function_call(prompt, "auto")
    response_message = response["choices"][0]["message"]

    if response_message.get("function_call"):
        print(response_message.get("function_call"))
        function_args = json.loads(response_message["function_call"]["arguments"])
        filter = create_filter_expression(function_args)
        print(filter)    

        messages.append(
            { 
                "role": response_message["role"], 
                "function_call" : {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"]
                },
                "content": None
            }
        )
    else:
        print("No function call")
        filter = ""
        
    result = vector_search(search_query, filter)
    
    messages.append(
        { 
            "role": "function", 
            "name": "search_results",
            "content": result
        }
    )
        
    response = openai.ChatCompletion.create(
        messages=messages,
        deployment_id=COMPLETIONS_MODEL,
        temperature=0.7,
    )

    return response['choices'][0]['message']    

In [5]:

# search_query = "¿cuántas horas se han dedicado a formación en el año 2022?"
# search_query = "¿cuántas certificaciones en el año 2022?"
search_query = "¿qué es pensar en colores?"
# search_query = "¿qué premios se recibieron en el año 2021?"
# search_query = "¿Cuántos MVP habían en el año 2018?"
# search_query = "¿Qué webinars se hicieron en el 2018?"

result = get_rag_response(search_query)
print(result)

No function call
0.87446976: memoria-social-encamina-2020.pdf#page=8: Tenemos un propósito  como empresa:- EQUIPO E NCAMINACrecer personal y profesionalmente,  inspirar y ayudar a otros, transformar   a nuestros clientes y crear valor para   el mundo.“«Pensar en  colores» es...“PENSAR EN COLORES” Es nuestra actitud fresca,  optimista y comprometida que utiliza el ingenio y la  creatividad para encontrar soluciones de tecnología  y talento que mejoren el presente de las personas, la  empresa y nuestra sociedad.   Una consultora debe Pensar en C olores porque la  tecnología y el conocimiento pueden llevarse mucho  mas lejos con una reflexión diferente, creativa, positiva y  comprometida.Impulsar el desarrollo del talento TIC .RSCSer líderes en el ecosistema Microsoft  para el ámbito  nacional y ser algún día, un referente internacional en  soluciones y servicios de nicho a través de la nube.  Y queremos lograrlo con intensidad, desde el lugar  donde más nos gusta vivir, con un equipo bri